In [ ]:
import cv2
import numpy as np
import os

: 

In [ ]:

def create_yolo_annotations(image_path, output_dir, class_id=0):
    """
    使用 OpenCV 检测物体轮廓并创建 YOLO 格式的标注文件
    
    参数:
    image_path: 输入图像的路径
    output_dir: 输出标注文件的目录
    class_id: 物体类别ID (默认为0)
    """
    # 读取图像
    image = cv2.imread(image_path)
    if image is None:
        print(f"无法读取图像: {image_path}")
        return
    
    # 获取图像尺寸
    height, width = image.shape[:2]
    
    # 转换为灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 应用高斯模糊减少噪声
    # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    blurred = gray
    
    # 使用Otsu's二值化
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 查找轮廓
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 创建标注文件
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    txt_path = os.path.join(output_dir, base_name + '.txt')
    
    with open(txt_path, 'w') as f:
        for contour in contours:
            # 计算最小外接矩形
            x, y, w, h = cv2.boundingRect(contour)
            
            # 过滤掉太小的轮廓（可选）
            if w < 10 or h < 10:
                continue
                
            # 转换为YOLO格式 (class_id, x_center, y_center, width, height)
            # YOLO格式使用相对坐标（相对于图像尺寸的比例）
            x_center = (x + w/2) / width
            y_center = (y + h/2) / height
            w_norm = w / width
            h_norm = h / height
            
            # 写入标注文件
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
            
            # 在图像上绘制矩形（用于可视化）
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # 显示结果（可选）
    cv2.imshow('Detected Objects', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    print(f"标注文件已保存到: {txt_path}")

# 使用示例
if __name__ == "__main__":
    # 设置输入图像路径和输出目录
    image_path = "path/to/your/image.jpg"
    output_dir = "path/to/output/directory"
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 处理单张图像
    create_yolo_annotations(image_path, output_dir)
    
    # 处理目录中的所有图像
    def process_directory(input_dir, output_dir):
        for filename in os.listdir(input_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(input_dir, filename)
                create_yolo_annotations(image_path, output_dir)
    
    # 处理整个目录
    # input_dir = "path/to/images/directory"
    # process_directory(input_dir, output_dir)